## Recommender System - Marketplace Matching

In this notebook, we will: 
- Clean textual data from user-input verbatim posts
- Use a Word2Vec model to calculate document similarities 
- Sort the most similar user input to our training data in order to recommend similar products 
- Save this model in a format that allows us to refresh the testing data

The goal of this project is to create a recommender system to help Pangeans find the right "project" for them, given their profile information. Here, we are using legacy data that is from Pangea V2, when Pangeans were allowed to post services and requests, as well as purchase items on the platform. We are using the User-Inputted Titles to suggest similar services, or in V3, similar "projects". 

In [1]:
#Importing Libraries
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords') ###

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

import os, sys

from operator import add

from sklearn.preprocessing import normalize

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from adjustText import adjust_text

from sklearn.manifold import TSNE

import pickle

import json
from pprint import pprint
from pandas.io.json import json_normalize

import operator

import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelateng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [3]:
def vectorize_and_store_existing_titles():
    
    raw = pd.read_csv("allPostData.csv", header=0);
    
    #we can replace this with a filepath in the future
    titles = raw['title'];
    post_titles = [title for title in titles];
    post_titles = set(post_titles);
    
    tokens = [[word for word in title.lower().split()] for title in post_titles];
    
    clean_words = [[word for word in title if word.isalpha()] for title in tokens];
    stoplist = set(stopwords.words('english'));
    
    titles_nostopwords = [[word for word in title if word not in stoplist] for title in clean_words];   
    #print(len(titles_nostopwords))
    
    filtered_word_list = [[word for word in title if word in model.vocab] for title in titles_nostopwords];
    #print(len(filtered_word_list))
    
    dictionary = dict(zip(post_titles, filtered_word_list))
    
    #title_vectors = {}
    
    #print(len(dictionary.keys()))
    
    #print(len(set(titles)))
    #print(len(set(post_titles)))
    #dupe reqs 
    
    vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
    #print(vectorized_titles)
    
    
    
    for title in post_titles: 
        word_vecs = [model[word] for word in dictionary[title]]
        #print(len(word_vecs))
        if len(word_vecs) == 0:
            title_vec = [np.zeros(300)]
        else: 
            title_vec = normalize(sum(word_vecs).reshape(1, -1))
            #print(title_vec)
       
        vectorized_titles = vectorized_titles.append({'Titles': title, 'Vectors': title_vec}, ignore_index=True)
       
    
    
    #dictionary = dict(zip(post_titles, filtered_word_list))

    #can also replace filepath in the future
    #titles.to_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/raw_titles.pkl")
    vectorized_titles.to_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
    #print(vectorized_titles)
    #print(title_vec)
    #print(word_vecs)
    return(vectorized_titles)

#vectorize_and_store_existing_titles()


In [4]:
#pd.read_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
#sanity check
vectorize_and_store_existing_titles()

,Titles,Vectors
0,CS15 Homework Help,"[[-0.011092936, 0.035716817, -0.008959679, 0.0..."
1,Looking For Factorio Players,"[[-0.0009527956, 0.11596884, 0.08493492, -0.01..."
2,music lessons!,"[[0.014816847, -0.08275069, -0.020408109, 0.01..."
3,SCRIPPS,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,Free Brownies,"[[-0.012820635, -0.035198472, 0.027884882, 0.1..."
5,I'll be friend,"[[0.02718549, -0.08211893, 0.058870647, 0.0361..."
6,Search Engine Optimizatio,"[[0.10572288, 0.0032037236, -0.035347752, -0.0..."
7,Spanish tutoring/conversation practice,"[[-0.025693027, -0.051538784, -0.004831761, 0...."
8,Buddy,"[[-0.0023477953, -0.055394985, 0.012810091, 0...."
9,Start Up Consulting,"[[-0.0074573867, 0.041665584, -0.008791506, 0...."


In [5]:
#can change this directory later


#test post 
with open('firstPost.json') as fresh_data:
    user_post = json.load(fresh_data)

#user_post = json.loads('firstPost.json')
#print(user_post)

#should only take in a single post at a time 

def vectorize_new_title(user_post):
    json_df = pd.DataFrame.from_dict(json_normalize(user_post), orient='columns')
    #print(json_df)
    
    title = json_df["title"]
    #print("this is the title", title)
    
    #json_post_titles = [title in title];
    json_post_titles = [title for title in title];
    json_tokens = [[word for word in title.lower().split()] for title in json_post_titles];
    #json_post_titles = set(json_post_titles)
    #print(json_post_titles)
    
    
    
    #json_clean_words = [[word for word in title if word.isalpha()] for title in json_post_titles];
    json_clean_words = [[word for word in title if word.isalpha()] for title in json_tokens];
    #print(json_clean_words)
    
    stoplist = set(stopwords.words('english'));
    json_titles_nostopwords = [[word for word in title if word not in stoplist] for title in json_clean_words];   
    
    
    json_filtered_word_list = [[word for word in title if word in model.vocab] for title in json_titles_nostopwords];
    #print(json_filtered_word_list)
    
    json_title_vectors = {}
    
    json_vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
    #print(json_vectorized_titles)
    
    
    for title in json_filtered_word_list: 
        json_word_vecs = [model[word] for word in title]
        if len(json_word_vecs) == 0:
            json_title_vec = [np.zeros(300)]
        else: 
            json_title_vec = normalize(sum(json_word_vecs).reshape(1, -1))
        
        json_vectorized_titles = json_vectorized_titles.append({'Titles': json_post_titles, 'Vectors': json_title_vec}, ignore_index=True)
    return(json_vectorized_titles)

#vectorize_new_title(json_df)
#why is it a dictionary in a dictionary?? double brackets

In [17]:
vectorized_title = vectorize_new_title(user_post)
#print(vectorized_title)

def rank_existing_titles(vectorized_title):
    #loop over all keys in dict 
    ranked_titles = {}
    #other_titles = pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/raw_titles.pkl")
    other_titles = pd.read_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
   
    #can also use title_vectors.keys() 
    for index,row in other_titles.iterrows():
        ranked_titles[row['Titles']] = np.dot(np.squeeze(np.asarray(row['Vectors'])), np.squeeze(np.asarray(vectorized_title['Vectors'][0])))
        #print(ranked_titles[row['Titles']])
        
    sorted_title_vecs = sorted(ranked_titles.items(), key=operator.itemgetter(1), reverse=True)
    return(sorted_title_vecs)

#rank_existing_titles(vectorized_title[:10])
#word querying

#should be cleaned titles x cleaned til

In [23]:
def generate_recommendations(user_post):
    title = user_post["title"]
    #print(title)
    vectorized_title = vectorize_new_title(user_post)
    #print(vectorized_title)
    ranked_titles = rank_existing_titles(vectorized_title)
    #ranked_titles = rank_existing_titles(title, vectorized_title)
    #print(ranked_titles)
    
    
    with open("/Users/angelateng/Dropbox/AMBER/SharpestMinds/ranked_titles.csv", "w", newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for title in ranked_titles:
            wr.writerow([ranked_titles, title])
    #csv_out = ranked_titles.to_csv('ranked_titles', encoding='utf-8', index=False)
    #writer = csv.writer(ranked_titles, delimiter='', quotechar='"', quoting=csv.QUOTE_ALL)
 
    #for row in reader:
        #writer.writerow(row)

       # with open('csv_out', 'wb') as csv_out:
         #   csv_writer = csv.writer(csv_out, quoting=csv.QUOTE_ALL)
         #   csv_writer.writerow('csv_out')
    return(ranked_titles)
    
    

#vectorize_and_store_existing_titles(); 
#vectorize_new_title(json_df);
#rank_existing_titles(vectorize_new_title);

In [24]:
#generate_recommendations(json_df)
pprint(generate_recommendations(user_post))

[('Teach Me How To Cook!', 1.0),
 ('Teach you how to shuffle', 0.72850794),
 ('Teach me how to cook quick meals!', 0.71625805),
 ('Teach me how to play piano', 0.71463776),
 ('Teach how to make tea', 0.69148445),
 ('Learn To Juggle!', 0.6698537),
 ('Teach how to prepare steaks ', 0.66081357),
 ('Help me learn Pop/Stars dance', 0.64242554),
 ('Teach Me How To Solve Rubiks Cube', 0.63249385),
 ('Teach me meme culture', 0.61345947),
 ('Orgo lesson ', 0.5588066),
 ('Math Tutor', 0.5531746),
 ('Math tutor', 0.5531746),
 ('Cooking Lessons', 0.5473081),
 ('Cooking lessons', 0.5473081),
 ('Cooking Lessons ', 0.5473081),
 ('Learn To Harmonize', 0.5445825),
 ('Amharic lessons ', 0.5328233),
 ('Fortnite Lessons', 0.5328233),
 ('Prototyping/Wireframing Lessons', 0.5328233),
 ('Self-Defense Lessons', 0.5328233),
 ('Mk8 Tutor ', 0.5296032),
 ('Splatoon Tutor', 0.5296032),
 ('Mariokart tutor', 0.5296032),
 ('Sm4sh Tutor', 0.5296032),
 ('Basic Painting Lessons', 0.52375805),
 ('Bosnian/Croatian Langua